# Read / Preprocess Data

In [71]:
import pandas as pd

In [72]:
# read dataset
data = pd.read_csv('compas-scores-two-years.csv', index_col=0)

## Remove unused columns

In [73]:
data = data[['age', 'c_charge_degree', 'race', 'age_cat', 'score_text', 'sex', 'priors_count', 
                    'days_b_screening_arrest', 'decile_score', 'two_year_recid', 'c_jail_in', 'c_jail_out']]

In [74]:
set(data['race'].values)

{'African-American',
 'Asian',
 'Caucasian',
 'Hispanic',
 'Native American',
 'Other'}

In [75]:
data = data[data['race'].apply(lambda race: race in {'African-American', 'Caucasian'})]
original_data = data.copy()

## Split target column

In [76]:
y_column = 'two_year_recid'

In [77]:
data_y = data[y_column]

In [78]:
data = data[[column for column in data.columns if column != y_column]]

## Convert date to number

In [79]:
date_columns = ['c_jail_in', 'c_jail_out']

In [80]:
for column in date_columns:
    data[column] = pd.to_datetime(data[column])
    data[column] = data[column] - data[column].min()
    data[column] = data[column].apply(lambda t: t.days)

In [81]:
data.dtypes

age                          int64
c_charge_degree             object
race                        object
age_cat                     object
score_text                  object
sex                         object
priors_count                 int64
days_b_screening_arrest    float64
decile_score                 int64
c_jail_in                  float64
c_jail_out                 float64
dtype: object

## One-hot encoding for categorical columns

In [82]:
from sklearn.preprocessing import OneHotEncoder

In [83]:
one_hot_encoder = OneHotEncoder(min_frequency=50)

In [84]:
categorical_columns = [
    'sex',
    'age_cat',
    'race',
    'c_charge_degree',
    'score_text',
]
categorical_x = one_hot_encoder.fit_transform(data[categorical_columns])

In [85]:
data = data.drop(columns=categorical_columns)

## Handle null values

In [86]:
for column in data.columns:
    if data[column].isna().any():
        data[column + '_null'] = data[column].isna()
        data[column] = data[column].fillna(0)
numerical_x = data.to_numpy(dtype=float)

## Generate Dataset

In [87]:
import numpy as np
from sklearn.model_selection import train_test_split

x = np.concatenate([numerical_x, categorical_x.toarray()], axis=1)
y = data_y.to_numpy()

In [88]:
indices = list(range(0, len(x)))
indices_train, indices_test = train_test_split(indices, random_state=42)

In [89]:
x_train, x_test, y_train, y_test = x[indices_train], x[indices_test], y[indices_train], y[indices_test]

In [90]:
data_train, data_test = original_data.iloc[indices_train], original_data.iloc[indices_test]

# Plain Model with Bias

## Train Plain Model

In [91]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=42)
model.fit(x_train, y_train)

C:\Users\Administrator\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [92]:
from sklearn.metrics import accuracy_score

pred_test = model.predict(x_test)
accuracy_score(pred_test, y_test)

0.7243172951885566

In [93]:
model.coef_.argmax()

12

In [94]:
x_train

array([[ 30.,   1.,  -1., ...,   0.,   1.,   0.],
       [ 21.,   0.,  -1., ...,   1.,   0.,   0.],
       [ 43.,   1.,  -1., ...,   0.,   1.,   0.],
       ...,
       [ 21.,   2., -10., ...,   0.,   0.,   1.],
       [ 31.,   6.,   0., ...,   0.,   1.,   0.],
       [ 42.,  13.,  -1., ...,   1.,   0.,   0.]])

## Bias Metrics

In [99]:
data_test['prediction'] = pred_test

C:\Users\Administrator\AppData\Local\Temp\ipykernel_58100\3939615259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['prediction'] = pred_test


In [100]:
def metrics_report(y_gt, y_pred):
    tp = ((y_gt == 1) & (y_pred == 1)).sum()
    fp = ((y_gt == 0) & (y_pred == 1)).sum()
    fn = ((y_gt == 1) & (y_pred == 0)).sum()
    tn = ((y_gt == 0) & (y_pred == 0)).sum()
    fnr = fn / (tp + fn)
    fpr = fp / (fp + tn)
    fdr = fp / (tp + fp)
    fomr = fn / (fn + tn)
    omr = (fn + fp) / (tp + fp + fn + tn)
    print('False Negative Rate', fnr)
    print('False Positive Rate', fpr)
    print('False Discovery Rate', fdr)
    print('False Omission Rate', fomr)
    print('Overall Misclass Rate', omr)

In [101]:
african_data = data_test[data_test['race'] == 'African-American']
metrics_report(african_data['two_year_recid'], african_data['prediction'])

False Negative Rate 0.2936344969199179
False Positive Rate 0.2631578947368421
False Discovery Rate 0.25054466230936817
False Omission Rate 0.30752688172043013
Overall Misclass Rate 0.2792207792207792


In [102]:
caucasian_data = data_test[data_test['race'] == 'Caucasian']
metrics_report(caucasian_data['two_year_recid'], caucasian_data['prediction'])

False Negative Rate 0.5
False Positive Rate 0.12299465240641712
False Discovery Rate 0.27710843373493976
False Omission Rate 0.26785714285714285
Overall Misclass Rate 0.2703583061889251


Caucasian group has higher False Negative Rate and lower False Positive Rate than African-American, which means the model biasedly tends to consider Caucasian group innocent.

# Unbiased Model

Use 1 / -1 as labels, as suggested in paper.

In [103]:
def L(theta):
    m = len(y_train)  # Number of training examples
    h = 1 / (1 + np.exp(-np.dot(x_train, theta)))  # Logistic function
    epsilon = 1e-10  # Small value to prevent log(0) errors
    
    # Cross-entropy loss function
    loss = -(1/m) * np.sum(y_train * np.log(h + epsilon) + (1 - y_train) * np.log(1 - h + epsilon))
    
    return loss

In [104]:
d0_indices = data_train['race'] == 'Caucasian'
d1_indices = data_train['race'] == 'African-American'
d0_x_train = x_train[d0_indices]
d1_x_train = x_train[d1_indices]
d0_y_train = y_train[d0_indices] * 2 - 1
d1_y_train = y_train[d1_indices] * 2 - 1

def g1(theta, x, y):
    d = x @ theta
    return np.minimum(y * d, 0)

def g2(theta, x, y):
    d = x @ theta
    return np.minimum((1 - y) / 2 * y * d, 0)

def g3(theta, x, y):
    d = x @ theta
    return np.minimum((1 + y) / 2 * y * d, 0)

def st(theta, g):
    n0 = d0_indices.sum()
    n1 = d1_indices.sum()
    n = n0 + n1
    return -n1 / n * g(theta, d0_x_train, d0_y_train).sum() \
        + n0 / n * g(theta, d1_x_train, d1_y_train).sum()

In [105]:
np.random.seed(42)
theta0 = np.zeros(x_train.shape[-1])

## Train Plain Biased Model without constraints

First try minimize a normal logistic regression to verify the correctness of L and optimization.

In [106]:
from scipy.optimize import minimize
theta = minimize(L, theta0).x

In [107]:
theta_biased = theta
accuracy_score((x_test @ theta_biased) >= 0, y_test)

0.7249674902470741

In [108]:
st(theta_biased, g1), st(theta_biased, g2), st(theta_biased, g3)

(14.235750529959802, -47.921137887987, 62.1568884179468)

In [109]:
data_test['biased-prediction'] = x_test @ theta_biased > 0

C:\Users\Administrator\AppData\Local\Temp\ipykernel_58100\1596178535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['biased-prediction'] = x_test @ theta_biased > 0


In [110]:
african_data = data_test[data_test['race'] == 'African-American']
metrics_report(african_data['two_year_recid'], african_data['biased-prediction'])

False Negative Rate 0.28952772073921973
False Positive Rate 0.2608695652173913
False Discovery Rate 0.24782608695652175
False Omission Rate 0.30387931034482757
Overall Misclass Rate 0.275974025974026


In [111]:
caucasian_data = data_test[data_test['race'] == 'Caucasian']
metrics_report(caucasian_data['two_year_recid'], caucasian_data['biased-prediction'])

False Negative Rate 0.5083333333333333
False Positive Rate 0.12299465240641712
False Discovery Rate 0.2804878048780488
False Omission Rate 0.27111111111111114
Overall Misclass Rate 0.2736156351791531


## Train with constraints

### Constraint 1 to minimize gap of overall misclassification rate

In [112]:
constraint = {'type': 'eq', 'fun': lambda theta: st(theta, g1)}
theta = minimize(L, theta0, constraints=constraint).x

C:\Users\Administrator\AppData\Local\Temp\ipykernel_58100\4128375151.py:3: RuntimeWarning: overflow encountered in exp
  h = 1 / (1 + np.exp(-np.dot(x_train, theta)))  # Logistic function


In [113]:
accuracy_score((x_test @ theta) >= 0, y_test)

0.7249674902470741

In [114]:
st(theta, g1), st(theta, g2), st(theta, g3)

(5.738343133998569e-11, -39.88734879924521, 39.88734879930254)

In [115]:
data_test['unbiased1-prediction'] = x_test @ theta > 0

C:\Users\Administrator\AppData\Local\Temp\ipykernel_58100\713827953.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['unbiased1-prediction'] = x_test @ theta > 0


In [116]:
african_data = data_test[data_test['race'] == 'African-American']
metrics_report(african_data['two_year_recid'], african_data['unbiased1-prediction'])

False Negative Rate 0.29774127310061604
False Positive Rate 0.2562929061784897
False Discovery Rate 0.24669603524229075
False Omission Rate 0.30851063829787234
Overall Misclass Rate 0.27813852813852813


In [117]:
caucasian_data = data_test[data_test['race'] == 'Caucasian']
metrics_report(caucasian_data['two_year_recid'], caucasian_data['unbiased1-prediction'])

False Negative Rate 0.475
False Positive Rate 0.13903743315508021
False Discovery Rate 0.29213483146067415
False Omission Rate 0.26146788990825687
Overall Misclass Rate 0.2703583061889251


### Constraint 2 to minimize gap of false positive rate

In [118]:
constraint = {'type': 'eq', 'fun': lambda theta: st(theta, g2)}
theta = minimize(L, theta0, constraints=constraint).x

C:\Users\Administrator\AppData\Local\Temp\ipykernel_58100\4128375151.py:3: RuntimeWarning: overflow encountered in exp
  h = 1 / (1 + np.exp(-np.dot(x_train, theta)))  # Logistic function


In [119]:
accuracy_score((x_test @ theta) >= 0, y_test)

0.7223667100130039

In [120]:
st(theta, g1), st(theta, g2), st(theta, g3)

(2.164416319776592, 8.023022246561595e-10, 2.164416318974247)

In [121]:
data_test['unbiased2-prediction'] = x_test @ theta > 0

C:\Users\Administrator\AppData\Local\Temp\ipykernel_58100\1202990414.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['unbiased2-prediction'] = x_test @ theta > 0


In [122]:
african_data = data_test[data_test['race'] == 'African-American']
metrics_report(african_data['two_year_recid'], african_data['unbiased2-prediction'])

False Negative Rate 0.32854209445585214
False Positive Rate 0.22654462242562928
False Discovery Rate 0.2323943661971831
False Omission Rate 0.321285140562249
Overall Misclass Rate 0.2803030303030303


In [123]:
caucasian_data = data_test[data_test['race'] == 'Caucasian']
metrics_report(caucasian_data['two_year_recid'], caucasian_data['unbiased2-prediction'])

False Negative Rate 0.42083333333333334
False Positive Rate 0.17914438502673796
False Discovery Rate 0.32524271844660196
False Omission Rate 0.24754901960784315
Overall Misclass Rate 0.2736156351791531


### Constraint 3 to minimize gap of false negative rate

In [124]:
constraint = {'type': 'eq', 'fun': lambda theta: st(theta, g3)}
theta = minimize(L, theta0, constraints=constraint).x

C:\Users\Administrator\AppData\Local\Temp\ipykernel_58100\4128375151.py:3: RuntimeWarning: overflow encountered in exp
  h = 1 / (1 + np.exp(-np.dot(x_train, theta)))  # Logistic function


In [125]:
accuracy_score((x_test @ theta) >= 0, y_test)

0.7230169050715215

In [126]:
st(theta, g1), st(theta, g2), st(theta, g3)

(-11.137701556722448, -11.137701555991171, -7.312621619348647e-10)

In [127]:
data_test['unbiased3-prediction'] = x_test @ theta > 0

C:\Users\Administrator\AppData\Local\Temp\ipykernel_58100\2167011796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['unbiased3-prediction'] = x_test @ theta > 0


In [128]:
african_data = data_test[data_test['race'] == 'African-American']
metrics_report(african_data['two_year_recid'], african_data['unbiased3-prediction'])

False Negative Rate 0.3326488706365503
False Positive Rate 0.21739130434782608
False Discovery Rate 0.2261904761904762
False Omission Rate 0.32142857142857145
Overall Misclass Rate 0.27813852813852813


In [129]:
caucasian_data = data_test[data_test['race'] == 'Caucasian']
metrics_report(caucasian_data['two_year_recid'], caucasian_data['unbiased3-prediction'])

False Negative Rate 0.44166666666666665
False Positive Rate 0.16844919786096257
False Discovery Rate 0.3197969543147208
False Omission Rate 0.2541966426858513
Overall Misclass Rate 0.2752442996742671


## Optimize with all the constraints

In [137]:
c = 0.0001

In [138]:
constraint = (
    {'type': 'ineq', 'fun': lambda theta: st(theta, g1) + c},
    {'type': 'ineq', 'fun': lambda theta: -st(theta, g1) + c},
    {'type': 'ineq', 'fun': lambda theta: st(theta, g2) + c},
    {'type': 'ineq', 'fun': lambda theta: -st(theta, g2) + c},
    {'type': 'ineq', 'fun': lambda theta: st(theta, g3) + c},
    {'type': 'ineq', 'fun': lambda theta: -st(theta, g3) + c}
)
theta = minimize(L, theta0, method='SLSQP', constraints=constraint).x

C:\Users\Administrator\AppData\Local\Temp\ipykernel_58100\4128375151.py:3: RuntimeWarning: overflow encountered in exp
  h = 1 / (1 + np.exp(-np.dot(x_train, theta)))  # Logistic function


In [139]:
accuracy_score((x_test @ theta) >= 0, y_test)

0.7204161248374512

In [140]:
st(theta, g1), st(theta, g2), st(theta, g3)

(2.2943567046240787e-08, -9.993007839170787e-05, 9.995302195875411e-05)

In [141]:
data_test['unbiased-prediction'] = x_test @ theta > 0

C:\Users\Administrator\AppData\Local\Temp\ipykernel_58100\2720298200.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test['unbiased-prediction'] = x_test @ theta > 0


In [142]:
african_data = data_test[data_test['race'] == 'African-American']
metrics_report(african_data['two_year_recid'], african_data['unbiased-prediction'])

False Negative Rate 0.3347022587268994
False Positive Rate 0.22654462242562928
False Discovery Rate 0.23404255319148937
False Omission Rate 0.3253493013972056
Overall Misclass Rate 0.28354978354978355


In [143]:
caucasian_data = data_test[data_test['race'] == 'Caucasian']
metrics_report(caucasian_data['two_year_recid'], caucasian_data['unbiased-prediction'])

False Negative Rate 0.425
False Positive Rate 0.17647058823529413
False Discovery Rate 0.3235294117647059
False Omission Rate 0.24878048780487805
Overall Misclass Rate 0.2736156351791531
